## Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da aula passada, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada.

Na camada de auto-atenção, deve-se implementar (vide slide 34):
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Camada de feed forward (2-layer MLP)

Instrucões:
- É necessário fazer duas implementações da camada de auto-atenção: uma usando laços (ineficiente, mas fácil de entender) e outra matricial (eficiente mas difícil de entender). Usar slide 36 como referência.

- Fazer um assert para garantir que o resultado das duas implementações é exatamente igual.

- No treinamento, usar apenas a implementação matricial.

## Imports

In [29]:
import os
import sys
import random
import torch.nn as nn
import torch.nn.functional as F
import time
import math
from sklearn.model_selection import train_test_split
from torch import nn

## Variáveis Globais e Inicialização

In [2]:
# Global variables

# Vocabulary
vocab_size = 5000
context_size = 5
pattern = r'\w+|[,;.:!?\']'

# Training
batch_size = 128
epochs = 10
lr = 0.1

# Model
embedding_dim = 256
hidden_dim = 128

In [3]:
# Colab environment
IN_COLAB = 'google.colab' in sys.modules

if (IN_COLAB):
    %pip install colorama

    # Google Drive
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    project_folder="/content/drive/MyDrive/Classes/IA024/Aula_2_3"
    os.chdir(project_folder)
    !ls -la

## Faz download e carrega o dataset

In [4]:
# Check if download is necessary
if not os.path.exists("67724.txt.utf-8"):
    print("Downloading Gutenberg texts")

    !wget https://www.gutenberg.org/ebooks/67724.txt.utf-8
    !wget https://www.gutenberg.org/ebooks/67725.txt.utf-8

In [5]:
text = open("67724.txt.utf-8","r").read()
text += open("67725.txt.utf-8","r").read()

paragraphs = text.split("\n\n")

len(paragraphs)

4969

In [6]:
# Checking the text
print(paragraphs[0])

The Project Gutenberg eBook of O Guarany: romance brazileiro, Vol. 1 (of 2)
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.


In [7]:
cleaned_paragraphs = [paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()]

# Print 5 random paragraphs
num_paragraphs = len(cleaned_paragraphs)
for i in range(0,5):
    idx = random.randrange(num_paragraphs)
    print(f"{cleaned_paragraphs[idx]}\n")

print("Number of paragraphs: " + str(num_paragraphs))

len(cleaned_paragraphs)

Então Pery viu uma luz que se aproximava; era Loredano e o seu companheiro.

Estendeu o oleo sobre a ferida, estancou o sangue e respirou.

--Deixará!... exclamou a menina, empallidecendo. Tu queres me abandonar?

Mais do que podia fiou de si o autor. Relendo a obra depois de annos, achou elle tão mau e incorrecto quando escrevera, que para bem corrigir, fora mister escrever de novo. Para tanto lhe carece o tempo e sobra o tedio de um labor ingrato.

Quando algum aventureiro por acaso transpunha esse circulo que o indio traçava com o olhar em redor de si, Pery na posição sobranceira em que se collocára o percebia immediatamente.

Number of paragraphs: 4892


4892

## Análise do dataset

In [8]:
# Conta as palavras no dataset
from collections import Counter
import re

def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(pattern, text.lower()))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)

12610

## Criando um vocabulário

In [9]:
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}

In [10]:
print(f"Most Frequent Words: {most_frequent_words[:10]}")
print(f"Vocabulary Size: {len(vocab)}")

Most Frequent Words: ['.', ',', 'a', 'que', 'o', 'de', 'e', 'se', ';', 'um']
Vocabulary Size: 5000


In [11]:
def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(pattern, sentence.lower())]

print(cleaned_paragraphs[20])
print(encode_sentence(cleaned_paragraphs[20], vocab))

 Publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever.
[0, 146, 4383, 23, 0, 2, 8, 50, 117, 276, 266, 2669, 13, 1071, 0, 2, 4, 193, 137, 287, 5, 2264, 8, 0, 3, 2672, 1]


## Classe do dataset

In [12]:
# Dataset class
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, paragraphs, vocab, context):
    self.paragraphs = paragraphs
    self.vocab = vocab
    self.context = context
    self.tokens, self.targets = self.setup()

  def __len__(self):
    return len(self.tokens)

  def __getitem__(self, idx):
    return torch.tensor(self.tokens[idx]), torch.tensor(self.targets[idx])
  
  def setup(self):
    tokens = []
    targets = []
    for paragraph in self.paragraphs:
      encoded = encode_sentence(paragraph, self.vocab)
      
      # If paragraph is smaller than the context, skip it.
      if len(encoded) < self.context + 1:
          continue

      for i in range(len(encoded) - self.context):
        tks = encoded[i:i+self.context]
        tgt = encoded[i+self.context]
        # Only add if there are no unknown tokens in both context and target.
        bad_token = 0
        if not (bad_token in tks or tgt == bad_token):
          tokens.append(tks)
          targets.append(tgt)
    return tokens, targets


In [13]:
# Train/Validation split
train_data, val_data = train_test_split(cleaned_paragraphs, test_size=0.2, random_state=18)

train_dataset = CustomDataset(train_data, vocab, context_size)
val_dataset = CustomDataset(val_data, vocab, context_size)

# Counting all Samples
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print()
print(f"Training dataset samples: {len(train_dataset)}")
print(f"Validation dataset samples: {len(val_dataset)}")

Training samples: 3913
Validation samples: 979

Training dataset samples: 59646
Validation dataset samples: 16215


In [14]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

sample = next(iter(train_loader))
print(sample)

[tensor([[   9,    5,   67,  863,  132],
        [ 523,    2,   16,   15,  313],
        [  64,    2,    7,   12,  774],
        [  60,    4,  296,  675,    7],
        [  73,    9,   33,   13, 2636],
        [   3,  112, 1109,   10, 1461],
        [ 590,   34,   21, 4856,    1],
        [ 183,  941,   36,   68,  689],
        [3296, 3297,  491,  140,   86],
        [ 378,  651,  484,  178,   33],
        [ 681,    9,    4,   93,  216],
        [  16,    4,    5,  378, 2557],
        [ 349,    7,  335,  355,    2],
        [  35,    1,    8,  193,    6],
        [   2,  706,    8, 1976, 1368],
        [   2,    7,   46,   70,    5],
        [   9,  197,  294,    6,  255],
        [   4,   15,  217,  169, 1797],
        [ 410,   42,  217,    2,   39],
        [ 112,  515,   10,  376, 3638],
        [ 640,    2,    7,  913,   93],
        [1036,    2,    7, 1238,    5],
        [ 890,  951,    6, 4428,    8],
        [   2,    7, 3662,  270,   11],
        [1437,  507,    1,  582,  342],

## Model

#### Implementação das camadas de self-attention (Loop e Matricial)

In [ ]:
# Positional Embedding - as described in "Attention is All You Need"
class PositionalEncoding(nn.Module):
    def __init__(self, max_sequence, embedding_dim):
        super().__init__()
        self.positional_encoding = torch.zeros(max_sequence, embedding_dim)
        
        # List of positions from 0..max_sequence in vector form
        position_list = torch.arange(0, max_sequence, dtype=torch.float)
        position_list = position_list.unsqueeze(1)

        div_term = torch.exp(torch.arange(0, embedding_dim, 2).float() * (-math.log(10000.0) / embedding_dim))

        self.positional_encoding[:, 0::2] = torch.sin(position_list * div_term)
        self.positional_encoding[:, 1::2] = torch.cos(position_list * div_term)
        self.positional_encoding = self.positional_encoding.unsqueeze(0)
    
    def forward(self, x):
        batch_size, seq_length, embedding_dim = x.size()
        positional_encoding = self.positional_encoding[:, :seq_length, :]
        positional_encoding = positional_encoding.to(x.device)
        # Position encoding is added to the input embeddings.
        return x + positional_encoding
    

In [26]:
# Adaptado da implementação do Ramon, muito elegante e traduziu bem o slide em aula.

# Loop Implementation with Linear Projections
class SelfAttention_Loop(nn.Module):
  def __init__(self, embedding_dim, vocab_size):
    super().__init__()
    self.embed_dim = embedding_dim
    self.WQ = nn.Linear(embedding_dim, embedding_dim)
    self.WK = nn.Linear(embedding_dim, embedding_dim)
    self.WV = nn.Linear(embedding_dim, embedding_dim)
    self.WO = nn.Linear(embedding_dim, vocab_size)

  def forward(self, seq):
    E = []  
    for Xq in seq:
        q = self.WQ(Xq)
        scores = []
        for Xk in seq:
            k = self.WK(Xk)
            score = torch.dot(q, k.transpose(-1,0))
            scores.append(score)

        scores_tensor = torch.tensor(scores)  
        probs         = scores_tensor.softmax(dim=-1)

        e = 0
        for xv, p in zip(seq, probs):
            v = self.WV(xv)
            e += v * p

        e = self.WO(e)
        E.append(e)

    return torch.stack(E)   

In [27]:
# Matrix Implementation
class SelfAttention_Matrix(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, inputs):
    scores = torch.matmul(inputs, inputs.transpose(1, 2))
    probs = F.softmax(scores, dim=-1)
    new_embedding = torch.matmul(probs, inputs)
    return new_embedding

### Teste das Camadas de Atenção

In [28]:
# Test data
sample = next(iter(train_loader))
input = sample[0]
target = sample[1]

embeddings = nn.Embedding(vocab_size+1, embedding_dim)
embeds = embeddings(input)

# Loop
attn_loop = SelfAttention_Loop(embedding_dim, vocab_size)
embeds_attn_loop = attn_loop(embeds)

# Matrix
attn_matrix = SelfAttention_Matrix(embedding_dim)
embeds_attn_matrix = attn_matrix(embeds)





RuntimeError: 1D tensors expected, but got 2D and 2D tensors

In [17]:
class BengioModel_SelfAttentionMatrix(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, h):
        super(BengioModel_SelfAttentionMatrix, self).__init__()
        self.embeddings = nn.Embedding(vocab_size+1, embedding_dim)
        self.attention = SelfAttention_Matrix()        
        self.linear1 = nn.Linear(context_size * embedding_dim, h)
        self.relu = torch.nn.ReLU()
        self.linear2 = nn.Linear(h, vocab_size+1)
        # Softmax to scale outputs
        self.logSoftMax = torch.nn.LogSoftmax(dim=1)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        x = torch.stack(torch.unbind(embeds, dim=1), dim=1)
        # Camada de autoatenção
        attention  = self.attention(x)
        # Flatten embeddings
        embeds = embeds.view(attention.size(0), -1)
        # Linear layer
        out = self.linear1(embeds)
        out = self.relu(out)
        # Second layer
        out = self.linear2(out)
        # Softmax output
        out = self.logSoftMax(out)
        return out

In [18]:
class BengioModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, h):
        super(BengioModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size+1, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, h)
        self.relu = torch.nn.ReLU()
        self.linear2 = nn.Linear(h, vocab_size+1)
        # Softmax to scale outputs
        self.logSoftMax = torch.nn.LogSoftmax(dim=1)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        # Flatten embeddings
        embeds = embeds.view(embeds.size(0), -1)
        # Linear layer
        out = self.linear1(embeds)
        out = self.relu(out)
        # Second layer
        out = self.linear2(out)
        # Softmax output
        out = self.logSoftMax(out)
        return out

## Funções de Treinamento e Avaliação do Modelo

In [19]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'The model has a total of {total_params:,} parameters.')

In [20]:
def initial_eval(model):
    # Initial Perplexity and Loss
    # Before training
    model.eval()

    loss = 0
    perp = 0

    with torch.no_grad():
        for inputs, targets in train_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss += criterion(outputs, targets).item()

    loss /= len(train_loader)
    perp = torch.exp(torch.tensor(loss))

    print(f'Initial Loss: {loss:.4f}')
    print(f'Initial Perplexity: {perp:.4f}')

In [21]:
def train(model, criterion, optimizer):
      # Training Loop
      model.train()
      for epoch in range(epochs):

            epoch_start = time.time()
            # Metrics
            epoch_loss = 0
            epoch_correct = 0
            epoch_samples = 0

            for inputs, targets in train_loader:
                  inputs = inputs.to(device)  # Move input data to the device
                  targets = targets.to(device)

                  # Forward pass
                  outputs = model(inputs)
                  loss = criterion(outputs, targets)

                  # Backward pass and optimization
                  optimizer.zero_grad()
                  loss.backward()
                  optimizer.step()

                  # Loss
                  epoch_loss += loss.item()

                  # Predicted
                  _, predicted = torch.max(outputs, 1)
                  epoch_correct += (predicted == targets).sum().item()
                  epoch_samples += targets.size(0)

            # Calculate average loss and accuracy for epoch
            avg_loss = epoch_loss / len(train_loader)
            acc = epoch_correct / epoch_samples

            # Perplexity
            perp = torch.exp(torch.tensor(avg_loss))

            epoch_end = time.time()
            epoch_time = epoch_end - epoch_start
            # Print epoch statistics
            print(f'Epoch [{epoch+1}/{epochs}], Time:{epoch_time:.2f}, Loss: {avg_loss:.4f}, Accuracy: {acc:.2f}%, Perplexity: {perp:.4f}')


In [22]:
def eval(model, criterion):
    model.eval()

    loss_sum = 0
    total_sum = 0
    correct_sum = 0
    eval_round = 0

    loss = 0
    perp = 0

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)      
            loss_sum += loss

            # Get the predicted labels
            _, predicted = torch.max(outputs, 1)

            total_sum += targets.size(0)
            correct_sum += (predicted == targets).sum().item()
            eval_round += 1

    # Calculate accuracy
    acc = 100 * correct_sum / total_sum

    # Calculate average perplexity
    average_loss = loss_sum / len(val_loader)
    average_perplexity = torch.exp(average_loss)

    print(f'Test Accuracy: {acc:.2f}%')
    print(f'Average Loss: {average_loss:.2f}')
    print(f'Average Perplexity: {average_perplexity:.2f}')

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## 1. Sem camada de atenção

In [24]:
model = BengioModel(vocab_size, embedding_dim, context_size, hidden_dim)
print("Model without Self Attention:")
print()
count_parameters(model)

# Cross Entropy
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr)

model.to(device)

print()
print("Training Start")
print()
train(model, criterion, optimizer)

print()
print("Evaluation Start")
print()
eval(model, criterion)

Model without Self Attention:

The model has a total of 2,089,353 parameters.

Training Start

Epoch [1/10], Time:5.60, Loss: 6.4382, Accuracy: 0.11%, Perplexity: 625.3005
Epoch [2/10], Time:2.73, Loss: 5.5752, Accuracy: 0.15%, Perplexity: 263.8004
Epoch [3/10], Time:2.63, Loss: 5.1926, Accuracy: 0.18%, Perplexity: 179.9422
Epoch [4/10], Time:2.76, Loss: 4.9045, Accuracy: 0.20%, Perplexity: 134.8988
Epoch [5/10], Time:2.99, Loss: 4.6613, Accuracy: 0.22%, Perplexity: 105.7733
Epoch [6/10], Time:3.02, Loss: 4.4423, Accuracy: 0.24%, Perplexity: 84.9738
Epoch [7/10], Time:2.98, Loss: 4.2416, Accuracy: 0.26%, Perplexity: 69.5224
Epoch [8/10], Time:2.97, Loss: 4.0489, Accuracy: 0.28%, Perplexity: 57.3353
Epoch [9/10], Time:2.97, Loss: 3.8673, Accuracy: 0.30%, Perplexity: 47.8147
Epoch [10/10], Time:2.95, Loss: 3.6919, Accuracy: 0.31%, Perplexity: 40.1215

Evaluation Start

Test Accuracy: 19.25%
Average Loss: 5.32
Average Perplexity: 204.40


## 2. Com camada de atenção

In [25]:
model_attn = BengioModel_SelfAttentionMatrix(vocab_size, embedding_dim, context_size, hidden_dim)
print("Model with Self Attention:")
print()
count_parameters(model_attn)

# Cross Entropy
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model_attn.parameters(), lr)

model_attn.to(device)

print()
print("Training Start")
print()
train(model_attn, criterion, optimizer)

print()
print("Evaluation Start")
print()
eval(model_attn, criterion)

Model with Self Attention:

The model has a total of 2,089,353 parameters.

Training Start

Epoch [1/10], Time:3.19, Loss: 6.4088, Accuracy: 0.11%, Perplexity: 607.1473
Epoch [2/10], Time:3.15, Loss: 5.5639, Accuracy: 0.15%, Perplexity: 260.8489
Epoch [3/10], Time:3.17, Loss: 5.1855, Accuracy: 0.18%, Perplexity: 178.6596
Epoch [4/10], Time:2.86, Loss: 4.8980, Accuracy: 0.20%, Perplexity: 134.0163
Epoch [5/10], Time:2.96, Loss: 4.6604, Accuracy: 0.22%, Perplexity: 105.6798
Epoch [6/10], Time:3.01, Loss: 4.4430, Accuracy: 0.24%, Perplexity: 85.0256
Epoch [7/10], Time:3.03, Loss: 4.2402, Accuracy: 0.26%, Perplexity: 69.4239
Epoch [8/10], Time:3.10, Loss: 4.0496, Accuracy: 0.28%, Perplexity: 57.3766
Epoch [9/10], Time:3.27, Loss: 3.8671, Accuracy: 0.30%, Perplexity: 47.8021
Epoch [10/10], Time:3.14, Loss: 3.6914, Accuracy: 0.32%, Perplexity: 40.0991

Evaluation Start

Test Accuracy: 19.53%
Average Loss: 5.31
Average Perplexity: 201.63
